<a href="https://colab.research.google.com/github/claratayaoti/capacitacao/blob/main/webscraping_do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install PyMuPDF

import requests
import io
import pymupdf as fitz  # PyMuPDF
from datetime import datetime
from datetime import date
import pandas as pd
import re

# Dicionário para converter o nome do mês para a forma abreviada em português
meses_portugues = {
    "01": "Jan", "02": "Fev", "03": "Mar", "04": "Abr",
    "05": "Mai", "06": "Jun", "07": "Jul", "08": "Ago",
    "09": "Set", "10": "Out", "11": "Nov", "12": "Dez"
}

# Obter a data de hoje
hoje = date.today()
ano = hoje.strftime("%Y")
mes_numero = hoje.strftime("%m")  # Exemplo: "02"
dia = hoje.strftime("%d")

def gerar_url_diario_oficial():
    """Gera a URL do Diário Oficial de hoje."""
    mes_abreviado = meses_portugues[mes_numero]  # Exemplo: "Fev"
    mes_extenso = f"{mes_numero}_{mes_abreviado}"  # Exemplo: "02_Fev"
    return f"https://diariooficial.niteroi.rj.gov.br/do/{ano}/{mes_extenso}/{dia}.pdf"

def extrair_texto_url(pdf_url):
    """Extrai texto de um PDF a partir de uma URL."""
    response = requests.get(pdf_url)
    response.raise_for_status()

    with io.BytesIO(response.content) as pdf_data:
        with fitz.open(stream=pdf_data, filetype="pdf") as doc:
            texto = ""
            for pagina in doc:
                texto += pagina.get_text()
            return texto

# Obter a URL do Diário Oficial
url_diario_oficial = gerar_url_diario_oficial()

# Extrair o texto do PDF
texto_diario_oficial = extrair_texto_url(url_diario_oficial)

# Dividir o texto em linhas
linhas = texto_diario_oficial.split('\n')

# Listas para armazenar os dados
decretos = []
portarias = []

# Padrões de busca
padrao_decreto = re.compile(r"DECRETO N° (\d+/\d+)")
padrao_portaria_nomeacao = re.compile(r"Port\. Nº (\d+/\d+)- Nomeia ([A-Za-zÀ-ÿ\s]+) para exercer o cargo de ([A-Za-zÀ-ÿ\s,]+), ([A-Za-zÀ-ÿ\s]+), (da|do) ([A-Za-zÀ-ÿ\s]+)")
padrao_portaria_exoneracao = re.compile(r"Port\. Nº (\d+/\d+)- (Exonera|Exonerar|Exonera, a pedido,|Exonerar, a pedido,) ([A-Za-zÀ-ÿ\s]+) do cargo de ([A-Za-zÀ-ÿ\s,]+), ([A-Za-zÀ-ÿ\s]+), (da|do) ([A-Za-zÀ-ÿ\s]+)")

# Percorrer as linhas do texto

for i, linha in enumerate(linhas):
    # Capturar decretos
    match = padrao_decreto.search(linha)
    if match:
        decreto_text = match.group(0)  # Start with the matched decree line
        j = i + 1  # Start from the next line
        while j < len(linhas) and "RODRIGO NEVES – PREFEITO" not in linhas[j]:
            decreto_text += "\n" + linhas[j]  # Append subsequent lines
            j += 1
        decretos.append(decreto_text)  # Append the entire decree text

    # Capturar portarias (nomeação e exoneração)
for padrao in [padrao_portaria_nomeacao, padrao_portaria_exoneracao]:
        match = padrao.search(linha)
        if match:
            numero_portaria = match.group(1)
            tipo = "Nomeação" if padrao is padrao_portaria_nomeacao else "Exoneração"
            nome = match.group(2).strip()
            cargo = match.group(3).strip()
            codigo_cargo = match.group(4).strip()
            orgao = match.group(5).strip()

            # Encontrar o texto até o ponto final após o nome
            texto_apos_nome = linha[linha.find(nome) + len(nome):]
            texto_ate_ponto = texto_apos_nome[:texto_apos_nome.find(".") + 1].strip()

            portarias.append([numero_portaria, tipo, nome, cargo, codigo_cargo, orgao, texto_ate_ponto])  # Adiciona o texto completo
            break  # Sai do loop interno se encontrar uma correspondência

# Criar DataFrame para os Decretos
df_decretos = pd.DataFrame(decretos, columns=["Texto Completo"])

# Criar DataFrame para as Portarias
df_portarias = pd.DataFrame(portarias, columns=["Número", "Tipo", "Nome", "Cargo", "Código", "Órgão"])

# Salvar como CSV
df_decretos.to_csv(f"{dia}_decretos.csv", index=False, encoding="utf-8")
df_portarias.to_csv(f"{dia}_portarias.csv", index=False, encoding="utf-8")

print("Arquivos CSV gerados: 'decretos.csv' e 'portarias.csv'")

Arquivos CSV gerados: 'decretos.csv' e 'portarias.csv'
